In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.style
import matplotlib as mpl

from common_functions import foo


mpl.style.use('classic')

%load_ext autoreload
%autoreload 2

results = "results"

In [3]:
from common_functions import latexify, format_axes, tableau20blind
latexify(14, 8)

In [4]:
lastKDays = 30
df = foo(lastKDays, results, True)
df.groupby(["reg"]).mean()

,kout,kin,day,rmse,lastKDays,seed
reg,,,,,,
gp,2.498016,2.001424,163.983420,20.665598,30.0,-1.000000
gp_rd,2.499959,1.999573,163.968984,24.380714,30.0,2.000305
knn,2.500000,2.000000,164.000000,30.679426,30.0,-1.000000
knn_rd,2.500000,2.000000,164.000000,30.541583,30.0,2.000000
qbc,2.500000,2.000000,164.000000,22.671395,30.0,-1.000000
qbc_rd,2.500000,2.000000,164.000000,24.136366,30.0,2.000000


In [21]:
regs = df["reg"].unique()
d = {}
for reg in regs:
    if reg == 'gp':
        continue
    td = {}
    for ix, tempdf in df.groupby(["kout", "kin"]):
        tempdf1 = tempdf.groupby("reg").mean()
#         print (tempdf1)
        gp_rmse = tempdf1.loc["gp"]["rmse"]
        reg_rmse = tempdf1.loc[reg]["rmse"]
#         print ("gp", reg)
#         print (gp_rmse, reg_rmse)
        td[ix] = 100*(reg_rmse - gp_rmse)/gp_rmse # want max
    d[reg] = td
    

In [22]:
d

{'qbc': {(0, 0): 20.239991001699266,
  (0, 1): 10.703729228562944,
  (0, 2): 29.301479380210438,
  (0, 3): 32.976837139192114,
  (0, 4): 13.575576176478815,
  (1, 0): -7.352603126617488,
  (1, 1): -20.621209195732355,
  (1, 2): 9.329435188960575,
  (1, 3): 14.749795060336828,
  (1, 4): 17.77212535765178,
  (2, 0): -2.192475568094852,
  (2, 1): -17.707096126129418,
  (2, 2): 15.32110310396753,
  (2, 3): 1.634686620178661,
  (2, 4): 52.2645837455235,
  (3, 0): 26.47718897529197,
  (3, 1): 17.83209653951583,
  (3, 2): 4.246900261573831,
  (3, 3): 20.764957493047476,
  (3, 4): 12.705651073852247,
  (4, 0): 16.511272602292237,
  (4, 1): 0.37351935338433406,
  (4, 2): 31.082046012962643,
  (4, 3): 25.87356182957577,
  (4, 4): 31.756858939106717,
  (5, 0): 3.9533619824122948,
  (5, 1): -24.49819133017378,
  (5, 2): 16.078657604558966,
  (5, 3): 0.6555534163499491,
  (5, 4): 11.161359867290807},
 'knn': {(0, 0): 64.51621857359048,
  (0, 1): 56.93914559807302,
  (0, 2): 82.28875893507153,
  (0,

In [23]:
new_d = {reg: max(d[reg].items(), key=lambda x:x[1]) for reg in d.keys()}
new_d

{'qbc': ((2, 4), 52.2645837455235),
 'knn': ((4, 4), 88.22409651241024),
 'qbc_rd': ((0, 2), 68.84430077885716),
 'gp_rd': ((4, 3), 50.543474913715),
 'knn_rd': ((0, 3), 94.05658759999409)}

In [6]:
tempdf1

,kout,kin,day,rmse,lastKDays,seed
reg,,,,,,
gp,0.0,0.0,164.000000,17.802756,30.0,-1.0
gp_rd,0.0,0.0,163.864717,15.528422,30.0,2.0
knn,0.0,0.0,164.000000,29.288422,30.0,-1.0
knn_rd,0.0,0.0,164.000000,29.887908,30.0,2.0
qbc,0.0,0.0,164.000000,21.406033,30.0,-1.0
qbc_rd,0.0,0.0,164.000000,22.579522,30.0,2.0


In [4]:
df.groupby(["reg"]).std()

,kout,kin,day,rmse,lastKDays,seed
reg,,,,,,
gp,1.707732,1.414644,95.057566,19.175092,0.0,0.000000
gp_rd,1.708196,1.414393,95.009321,24.014840,0.0,1.414551
knn,1.707912,1.414285,94.978492,21.823182,0.0,0.000000
knn_rd,1.707842,1.414228,94.974643,21.602620,0.0,1.414228
qbc,1.707912,1.414285,94.978492,19.006829,0.0,0.000000
qbc_rd,1.707842,1.414228,94.974643,19.142910,0.0,1.414228


In [5]:
splits = 6
dic = {}
for lastKDays in [30]:
    tdic = {}
    df = foo(lastKDays, results, True)
    days = df['day'].unique()
    for ix, tempdf in df.groupby(["kout", "kin"]):
        for reg_comp in ["qbc", "knn", "qbc_rd", "gp_rd", "knn_rd"]:
            comp = reg_comp
#             try:
            a =  tempdf[tempdf["reg"] == 'gp']["rmse"].values
            if sum(np.isnan(a)) != 0:
                raise ValueError
            b = tempdf[tempdf["reg"] == reg_comp]["rmse"].values
#             print (b)
            if sum(np.isnan(b)) != 0:
                raise ValueError
            a = a.mean()
            b = b.mean()
            if a <= b:
                temp = tdic.get(comp, [0, 0])
                tdic[comp] = [temp[0] + 1, temp[1]]
            else:
                temp = tdic.get(comp, [0, 0])
                tdic[comp] = [temp[0], temp[1] + 1]
#             except Exception as e:
#                 print (e)
#                 continue
    print ("lastKDays Done: ", lastKDays)
    dic[f"lastKDays_{lastKDays}"] = tdic

lastKDays Done:  30


In [6]:
dic

{'lastKDays_30': {'qbc': [25, 5],
  'knn': [29, 1],
  'qbc_rd': [26, 4],
  'gp_rd': [24, 6],
  'knn_rd': [30, 0]}}

In [7]:
new_dic = {
    'lastKDays': [],
    'knn': [],
    'knn_rd': [],
    'qbc': [],
    'qbc_rd': [],
    'gp_rd': [],
}
for ls, di in dic.items():
    new_dic['lastKDays'].append(ls)
    new_dic['knn'].append(tuple(di['knn']))
    new_dic['knn_rd'].append(tuple(di['knn_rd']))
    new_dic['qbc'].append(tuple(di['qbc']))
    new_dic['qbc_rd'].append(tuple(di['qbc_rd']))
    new_dic['gp_rd'].append(tuple(di['gp_rd']))

In [8]:
win_loss = pd.DataFrame(new_dic)
win_loss

,lastKDays,knn,knn_rd,qbc,qbc_rd,gp_rd
0,lastKDays_30,"(29, 1)","(30, 0)","(25, 5)","(26, 4)","(24, 6)"


In [9]:
dic = {
    "lastKDays": [] ,
    "best_ix": [] ,
    "worst_ix": [] ,
    "comp": [],
}
try:
    win_loss_plots = pd.read_csv("win_loss_plots_combine.csv")
except:
    for comp in ["qbc", "qbc_rd", "gp_rd", 'knn', 'knn_rd']:
        splits = 6
        import sys
        max_err = sys.maxsize
        min_err = - sys.maxsize
        for lastKDays in [30]:
            tdic = {}
            df = foo(lastKDays, results, True)
            days = df['day'].unique()
            best_ratio = 0
            worst_ratio = 1
            best_ix = None
            worst_ix = None
            for ix, tempdf in df.groupby(["kout", "kin"]):
                gp_win = 0
                gp_loose = 0
                for jx, tempdf1 in tempdf.groupby("day"):
                    a = tempdf1[tempdf1["reg"] == 'gp']["rmse"]
                    b = tempdf1[tempdf1["reg"] == comp]["rmse"]
                    if (a.shape[0] != 0 and b.shape[0] != 0):
                        a = a.mean()
                        b = b.mean()
                    else:
                        print (f"{jx}th Day did not have values for both")
                        continue

                    
                    if a <= b:
                        gp_win += 1
                    else:
                        gp_loose += 1

                win_ratio = float(gp_win)/(gp_loose + gp_win)
                if best_ratio < win_ratio:
                    best_ratio = win_ratio
                    best_ix = ix
                if worst_ratio > win_ratio:
                    worst_ratio = win_ratio
                    worst_ix = ix
            print (comp, "lastKDays Done: ", lastKDays)
            dic["lastKDays"].append(lastKDays)
            dic["best_ix"].append(best_ix)
            dic["worst_ix"].append(worst_ix)
            dic["comp"].append(comp)
            
    win_loss_plots = pd.DataFrame(dic)


In [10]:
win_loss_plots.to_csv("win_loss_plots_combine.csv", index = None)
win_loss_plots["best_ix"].values
win_loss_plots
win_loss_plots = pd.read_csv("win_loss_plots_combine.csv")
win_loss_plots

,lastKDays,best_ix,worst_ix,comp
0,30,"(2, 4)","(5, 1)",qbc
1,30,"(0, 2)","(5, 1)",qbc_rd
2,30,"(0, 3)","(0, 0)",gp_rd
3,30,"(0, 2)","(1, 1)",knn
4,30,"(0, 3)","(5, 1)",knn_rd
